# <span style="color:blue"> 이토록 쉬운 통계 & R </span>

<img src="../image/cover.png" width="200" height="400" align="left">  임경덕 지음

## PART 4 데이터 분석 도구, R

## 95% R로 분석 다시보기-2-treat-4-선형대수기초 대응분석관련-11 (51일차)

   * 데이타프로젝트 (http://dataproject.xyz) 또는 (http://cafe.naver.com/dataartproject) 에 미리 저장된 스크립트를 다운 받을 수 있습니다.
   * 스크립트를 활용하면 직접 명령어를 입력하지 않고 저장된 명령어들을 실행할 수 있습니다.

### 데이터 파일은 dataartproject.xyz 에 있습니다.

# 두개의 범주형 변수의 관계 찾기 -- treat - 대응분석 정리

## 카이제곱 분포

대응분석을 설명하기전에 먼저 카이제곱 분포에 대해서 먼저 설명하고자 합니다. 왜냐하면, 카이제곱 분포는 두개 이상의 범주형 변수간의 관계가 있는 것인지 또는 독립적인 것인지를 파악하기 위해서 사용하는 기법입니다.  
대응분석 역시 두개 이상의 범주형 변수간 관계가 있는 지를 그래프로 살피는 것이라서 카이제곱분포에 와 관계가 있습니다. 먼저 책 197 페이지에 있는 올림픽 대회별 획득한 메달이 대회별이라는 범주형 변수와 메달 색깔 독립형 변수와  관계가 있는 것인지 먼저 카이제곱 분포를 이용하여 살펴보고자 합니다.

또한 이것은 이전에 살펴보았던 정방형 행렬에서 고유값 분해를 직사각형 행렬 n x m 에서 특이값 분해 (SVD Singular Value Decomposition) 과 어떤 관계로 연결되는 지 살펴 보기로 합니다.

카이제곱 을 구하기 위해서는 다음의 절차를 진행합니다.
1. 관측값 행렬을 구합니다. O (Observed) 행렬
2. 예상값 행렬을 구합니다. E (Expected) 행렬
3. 잔차 행렬을 구합니다. R = O - E (Residual)
4. 책에서는 R^2/E 의 행렬 합을 구함으로서 카이제곱을 구합니다. 그러나 SVD 를 구하기 위해서는 지수잔차 행렬 indexed resudial 를 먼저 구합니다.
5. 표준잔차 행렬을 구합니다. Z (Standard Redidual)
6. 카이제곱을 구합니다.
7. SVD 를 구합니다.
8. 주축방향 을 구합니다. (이부분으 PCA 의 Dim1, Dim2 관련된 부분입니다.)

먼저 책 90 페이지에 있는 메달수 교차표를  다음과 같이 구합니다.

In [1]:
# 99 페이지에 있는 올림픽 대회별 메달수 
Medal = matrix(c(9, 13, 13, 9, 12, 10, 8, 3, 9, 9, 7, 9), 
    ncol = 3,
    dimnames = list( "올림픽대회" = c("28회 아테네", "29회 베이징", "30회 런던", "31회 리우"),
                         "메달" =  c("금메달", "은메달", "동메달")
                    )
    )  
Medal

,금메달,은메달,동메달
28회 아테네,9,12,9
29회 베이징,13,10,9
30회 런던,13,8,7
31회 리우,9,3,9


메달값을 계산하기 쉽게 하기 위해서 table 구조로 변화합니다.

In [2]:
# 교차표를 테이블 형식으로 변환
Medal_O <- as.table(Medal)
Medal_O

             메달
올림픽대회 금메달 은메달 동메달
  28회 아테네      9     12      9
  29회 베이징     13     10      9
  30회 런던       13      8      7
  31회 리우        9      3      9

In [3]:
# 단위는 메달수입니다. 계산 편의상 단위를 없애기 위해 각 행렬의 셀에 있는 메달수를 전체 메달수로 나누어 줍니다.
# 전체 메달수
Medal_T <- sum(Medal_O)
Medal_T

[1] 111

In [4]:
# 전체 메달수로 나눈 테이블은 다음과 같습니다.
MedalP_O <- Medal_O/Medal_T
MedalP_O

             메달
올림픽대회     금메달     은메달     동메달
  28회 아테네 0.08108108 0.10810811 0.08108108
  29회 베이징 0.11711712 0.09009009 0.08108108
  30회 런던   0.11711712 0.07207207 0.06306306
  31회 리우   0.08108108 0.02702703 0.08108108

In [5]:
# 전체로 나누었으므로 합을 구하면 1 이 될 것입니다.
sum(MedalP_O)

[1] 1

In [7]:
# 예상값 테이블을 구하기 위해서  각 주변값 벡터를 구합니다. 이것을 구하는 방법은 여러가지 rowSums(), colSums() 가 있으나 
# 여기서는 margin.table 함수를 이용합니다.
# row 별 합 또는 rowMass 라고 합니다.
MedalPMR_O <- margin.table(MedalP_O, 1)
# col 별 합 또는 columMass 라고 합니다.
MedalPMC_O <- margin.table(MedalP_O, 2)

# 각각의 값을 보면
cat("row 별 합=", MedalPMR_O, "\n\n")
cat("col 별 합=", MedalPMC_O, "\n\n")

row 별 합= 0.2702703 0.2882883 0.2522523 0.1891892 

col 별 합= 0.3963964 0.2972973 0.3063063 



In [11]:
# 이들은 rowSum 함수를 이용해서 구한것 하고는 같은 값입니다. 여기에 메달 총수를 곱하면 메달수가 나옵니다.
rowSums(MedalP_O, 1)
rowSums(MedalP_O, 1) * Medal_T
colSums(MedalP_O, 2)
colSums(MedalP_O, 2) * Medal_T

28회 아테네 29회 베이징   30회 런던   31회 리우 
  0.2702703   0.2882883   0.2522523   0.1891892

28회 아테네 29회 베이징   30회 런던   31회 리우 
         30          32          28          21

금메달    은메달    동메달 
0.3963964 0.2972973 0.3063063

금메달 은메달 동메달 
    44     33     34

In [13]:
# 예상갑 행렬은 row 별합 벡터와 col별 합 벡터을 outer 곱을 하거나 또는 col 의 transpose 행렬곱을 하면 구해집니다.
# 행렬 outer 곱을 하면
MedalP_E <- MedalPMR_O %o% MedalPMC_O
MedalP_E

,금메달,은메달,동메달
28회 아테네,0.10713416,0.08035062,0.08278549
29회 베이징,0.11427644,0.08570733,0.08830452
30회 런던,0.09999188,0.07499391,0.07726646
31회 리우,0.07499391,0.05624543,0.05794984


In [16]:
# 여기에 총 메달수를 곱하면 예상 책 189 페이지에 있는 예상 메달수 가 나옵니다.
# 책에서는 소숫점을 표시 하지 않았으므로 소숫점 이하를 round off 합니다.
round(MedalP_E * Medal_T,0)

,금메달,은메달,동메달
28회 아테네,12,9,9
29회 베이징,13,10,10
30회 런던,11,8,9
31회 리우,8,6,6


In [18]:
# 참고로 행렬곱을 이용하려면 아래와 같은 연산자를 사용하며 같은 결과가 나옵니다.
MedalPMR_O %*% t(MedalPMC_O)

,금메달,은메달,동메달
28회 아테네,0.10713416,0.08035062,0.08278549
29회 베이징,0.11427644,0.08570733,0.08830452
30회 런던,0.09999188,0.07499391,0.07726646
31회 리우,0.07499391,0.05624543,0.05794984


In [20]:
# 이제 잔차 Residuals 테이블을 구하도록 합니다.
# R = O - E 이므로
MedalP_R <- MedalP_O - MedalP_E
MedalP_R

             메달
올림픽대회       금메달       은메달       동메달
  28회 아테네 -0.026053080  0.027757487 -0.001704407
  29회 베이징  0.002840679  0.004382761 -0.007223440
  30회 런던    0.017125233 -0.002921841 -0.014203393
  31회 리우    0.006087168 -0.029218408  0.023131239

In [21]:
# 마찬가지로 여기에 전체 메달수를 곱하면 페이지 189 에 있는 차이값이 나옵니다. 정수로 표시하기 위해 소숫점을 round off 하면
round(MedalP_R * Medal_T, 0)

             메달
올림픽대회 금메달 은메달 동메달
  28회 아테네     -3      3      0
  29회 베이징      0      0     -1
  30회 런던        2      0     -2
  31회 리우        1     -3      3

In [22]:
# 이제 잔차지수(indexed redidual) 테이블을 구하도록 합니다. 이것은 (O - E)/E 로 관측값을 예상값으로 뺀 후에 예상값으로 나눈것을 말합니다.
# 지수라고 표현하는 것은 잔차를 예상값으로 나누어서 단위를 없앴기 때문입니다.
MedalP_I <- MedalP_R / MedalP_E
MedalP_I

             메달
올림픽대회      금메달      은메달      동메달
  28회 아테네 -0.24318182  0.34545455 -0.02058824
  29회 베이징  0.02485795  0.05113636 -0.08180147
  30회 런던    0.17126623 -0.03896104 -0.18382353
  31회 리우    0.08116883 -0.51948052  0.39915966

In [26]:
# 이제 표준잔차를 구합니다. 표준잔차를 이용하여 SVD 를 구하려고 하기 때문입니다.
# 사실은 표준잔차의 제곱에 전체 매달수를 곱하여 행렬의 모든 요소를 더하면 카이제곱이 나옵니다. 이제 이것을 보일려고 합니다.
# 표준잔차 Z = I * sqrt(E) 입니다. 
# 이것은 Z = I * sqrt(E) = R/E * sqrt(E) = R/sqrt(E) = (O - E)/sqer(E)
# 그러므로 Z^2 = (O - E)^2 / E 이므로 이것의 모든 합은 카이제곱이 됩니다.
MedalP_Z <- MedalP_I * sqrt(MedalP_E)
MedalP_Z

             메달
올림픽대회       금메달       은메달       동메달
  28회 아테네 -0.079596705  0.097923185 -0.005923743
  29회 베이징  0.008403181  0.014970587 -0.024308188
  30회 런던    0.054156941 -0.010669487 -0.051097140
  31회 리우    0.022228098 -0.123200623  0.096088801

In [28]:
# 이제 카이제곱을 구하도록 합시다.
# 카이제곱 = 표준잔차^2 * 총메달수 이므로
# 페이지 191 에 있는 카이제곱과 비숫한 값(5.78)이 나옵니다. 값이 다른 이유는 책에서는 소숫점을 반올림 하여 계산한 결과이기 떄문입니다.
chisq <- MedalP_Z^2 * Medal_T
sum(chisq)

[1] 5.262351

In [32]:
# R 에서는 이렇게 복잡한 과정을 거치지 않고 바로 구할 수 있는 함수가 있습니다. chisq.test() 를 수행하면 같은 결과 가 나옵니다.
# P 값도 같이 나오는데.. 0.5106 으로 유의값 0.05 보다 많이 크므로.. 영가설을 기각할 수 없고.. 대회와 메달수는 독립적이라고 할 수 있습니다.
# 즉 위 데이터는 일반적으로 독립적으로 분포된 모집단에서 얼마든지 나올수 있는 표본입니다.
chisq.test(Medal)


	Pearson's Chi-squared test

data:  Medal
X-squared = 5.2624, df = 6, p-value = 0.5106


In [37]:
# SVD 는 표준잔차에서 시작합니다. 이것을 하는 이유는 PCA 처럼 주성분을 구하고 그것을 그래프로 표시, 두 범주의 각 수준 사이의 거리가 어느 정도 
# 가까운지 표시 하는 방법입니다. 다음에는 그 것이 대한 설명을 하기 전에 먼저 SVD 하면 어떤 결과가 있는지 살펴 보아요.
# SVD 는 svd() 함수를 이용합니다.
SVD = svd(MedalP_Z)

In [39]:
rownames(SVD$u) <- rownames(MedalP_O)
rownames(SVD$v) <- colnames(MedalP_O)

In [40]:
SVD

28회 아테네,-0.58927797,0.5507930,-0.5202387
29회 베이징,-0.10567911,-0.2055297,-0.5358747
30회 런던,0.02973776,-0.7006122,-0.5029014
31회 리우,0.80043680,0.4043850,-0.4350635
금메달,0.3450432,-0.69609538,-0.6296002
은메달,-0.8344935,0.07951932,-0.5452498
동메달,0.4296112,0.71353198,-0.5534495


SVD 에는 3개의 성분이 있습니다.  
d: 제곱이 특이값이 되며, 제곱을 하여 더한후에 전체 메달수를 곱하면 역시 카이제곱이 나옵니다.  
u: left singular vector 행렬입니다.  
d: right singular vector 행렬입니다. 이부분이 이전에 정리된 자료를 찾으면 됩니다.  

In [43]:
# 먼저 고유값이 카이제곱이 되는지 확인합시다. --> 같은 값이 나옵니다.
eigenvalue <- SVD$d^2
eigenvalue

[1] 3.594711e-02 1.146146e-02 6.203422e-34

In [45]:
chi <- sum(eigenvalue) * Medal_T
chi

[1] 5.262351

In [49]:
# 좌표 메트릭스를 먼저 구하자.
# 먼저 row.masses 와 column.massed 를 구하면
row.masses <- MedalPMR_O
column.masses <- MedalPMC_O

In [50]:
# row 좌표 행렬은
standard.coordinates.rows = sweep(SVD$u, 1, sqrt(row.masses), "/")
standard.coordinates.rows

28회 아테네,-1.13349880,1.0594715,-1.0006991
29회 베이징,-0.19682291,-0.3827904,-0.9980441
30회 런던,0.05920941,-1.3949548,-1.0013026
31회 리우,1.84025874,0.9297087,-1.0002405


In [51]:
# column 좌표 행렬은
standard.coordinates.columns = sweep(SVD$v, 1, sqrt(column.masses), "/")
standard.coordinates.columns

금메달,0.5480354,-1.1056150,-1
은메달,-1.5304793,0.1458402,-1
동메달,0.7762429,1.2892451,-1


즉 이들 각각 범주형 변수는 2차원 그래표 상에 표현될 것이다. 이것은 다음시간에 설명할 것입니다.

In [52]:
# 주성분 좌표
# 주성분 좌표는 표준좌표에 singular value 를 곱해서 구해집니다.
principal.coordinates.rows = sweep(standard.coordinates.rows, 2, SVD$d, "*")
principal.coordinates.rows

28회 아테네,-0.21490823,0.11342510,-2.492408e-17
29회 베이징,-0.03731708,-0.04098085,-2.485795e-17
30회 런던,0.01122594,-0.14934134,-2.493911e-17
31회 리우,0.34890796,0.09953293,-2.491266e-17


In [53]:
principal.coordinates.columns = sweep(standard.coordinates.columns, 2, SVD$d, "*")
principal.coordinates.columns

금메달,0.1039060,-0.11836514,-2.490667e-17
은메달,-0.2901746,0.01561339,-2.490667e-17
동메달,0.1471735,0.13802425,-2.490667e-17


In [54]:
# 범주형 변수들의 좌표는 다음과 같이 구해 집니다.
pc = rbind(principal.coordinates.rows, principal.coordinates.columns) 
prop.table(pc ^ 2, 1) 

28회 아테네,0.782132438,0.21786756,1.051992e-32
29회 베이징,0.453309584,0.54669042,2.011455e-31
30회 런던,0.005618732,0.99438127,2.773031e-32
31회 리우,0.924745250,0.07525475,4.714553e-33
금메달,0.435222319,0.56477768,2.500698e-32
은메달,0.997113180,0.00288682,7.346100e-33
동메달,0.532047424,0.46795258,1.523780e-32


다음에는 대응분석관련 그래픽 기법에 대해서 설명합니다.